In [167]:
import pandas as pd

In [168]:
df = pd.read_csv("2016_General.csv")

In [169]:
df.head()

,Precinct,Race,LEG,CC,CG,CounterGroup,Party,CounterType,SumOfCount
0,ADAIR,Advisory Vote 14,45.0,3.0,1,Total,NP,Maintained,183
1,ADAIR,Advisory Vote 14,45.0,3.0,1,Total,NP,Registered Voters,519
2,ADAIR,Advisory Vote 14,45.0,3.0,1,Total,NP,Repealed,251
3,ADAIR,Advisory Vote 14,45.0,3.0,1,Total,NP,Times Blank Voted,51
4,ADAIR,Advisory Vote 14,45.0,3.0,1,Total,NP,Times Counted,485


In [170]:
selected = df[(df.CounterType == "Times Counted")]

In [171]:
selected.head()

,Precinct,Race,LEG,CC,CG,CounterGroup,Party,CounterType,SumOfCount
4,ADAIR,Advisory Vote 14,45.0,3.0,1,Total,NP,Times Counted,485
10,ADAIR,Advisory Vote 15,45.0,3.0,1,Total,NP,Times Counted,485
16,ADAIR,Attorney General,45.0,3.0,1,Total,NP,Times Counted,485
22,ADAIR,Commissioner of Public Lands,45.0,3.0,1,Total,NP,Times Counted,485
29,ADAIR,Congressional District 1,45.0,3.0,1,Total,NP,Times Counted,485


In [172]:
countdict = {}

In [173]:
for race in selected.Race:
    countdict.setdefault(race, 0)
    countdict[race] += 1

In [174]:
total_precincts = max(countdict.values())

county_wide_races = [race for race, count in countdict.items() if count == total_precincts]

In [175]:
only_countywide_races = df[df.Race.isin(county_wide_races)]

In [176]:
only_countywide_races = only_countywide_races.drop(columns = ["LEG", "CC", "CG", "CounterGroup", "Party"])

In [177]:
only_countywide_races.head()

,Precinct,Race,CounterType,SumOfCount
0,ADAIR,Advisory Vote 14,Maintained,183
1,ADAIR,Advisory Vote 14,Registered Voters,519
2,ADAIR,Advisory Vote 14,Repealed,251
3,ADAIR,Advisory Vote 14,Times Blank Voted,51
4,ADAIR,Advisory Vote 14,Times Counted,485


In [178]:
only_countywide_races["Race_Type"] = only_countywide_races["Race"] + ": " + only_countywide_races["CounterType"]

In [179]:
only_countywide_races = only_countywide_races.drop(columns = ["Race", "CounterType"])

In [180]:
only_countywide_races.head()

,Precinct,SumOfCount,Race_Type
0,ADAIR,183,Advisory Vote 14: Maintained
1,ADAIR,519,Advisory Vote 14: Registered Voters
2,ADAIR,251,Advisory Vote 14: Repealed
3,ADAIR,51,Advisory Vote 14: Times Blank Voted
4,ADAIR,485,Advisory Vote 14: Times Counted


In [181]:
only_countywide_races = only_countywide_races.pivot(index='Precinct', columns='Race_Type', values='SumOfCount')

In [182]:
only_countywide_races["Registered_Voters"] = only_countywide_races["Advisory Vote 14: Registered Voters"]

In [183]:
only_countywide_races["Total_Voters"] = only_countywide_races["Advisory Vote 14: Times Counted"]

In [184]:
only_countywide_races = only_countywide_races.drop(columns = [column for column in only_countywide_races.columns if "Registered Voters" in column]) 
only_countywide_races = only_countywide_races.drop(columns = [column for column in only_countywide_races.columns if "Times Counted" in column]) 

In [185]:
cols = only_countywide_races.columns.tolist()
cols = cols[-2:] + cols[:-2]
only_countywide_races = only_countywide_races[cols]

In [186]:
cols = only_countywide_races.columns.tolist()
import string
def encode(n):
    ''' Convert integer n to string.ascii_lowercase'''
    if n == 0:
        return "a"
    out = []
    while n:
        n, r = n // 26, n % 26
        out.append(string.ascii_lowercase[r])
    return ''.join(out)


In [203]:
data_dict = {column_name: encode(i) for i, column_name in enumerate(cols)}

In [188]:
only_countywide_races = only_countywide_races.rename(index = str, columns = data_dict)

In [189]:
#only_countywide_races.to_csv("csv2016.csv")

In [201]:
with open("data_dictionary.csv", "w+") as fh:
    fh.write("O.G. value, encoded as\n")
    for k, v in data_dict.items():
        fh.write(f'{k}, {v}\n')

In [215]:
## build dropdown option menu
for column_name, encoded_value in data_dict.items():
    ballot_name = " ".join("".join(column_name.replace("&","").replace("_",":").split(":")[1:2]).split()[:3])
    print(f'<option value="{encoded_value};{ballot_name}">{column_name}</option>')



<option value="a;Voters">Registered_Voters</option>
<option value="b;Voters">Total_Voters</option>
<option value="c;Maintained">Advisory Vote 14: Maintained</option>
<option value="d;Repealed">Advisory Vote 14: Repealed</option>
<option value="e;Times Blank Voted">Advisory Vote 14: Times Blank Voted</option>
<option value="f;Times Over Voted">Advisory Vote 14: Times Over Voted</option>
<option value="g;Maintained">Advisory Vote 15: Maintained</option>
<option value="h;Repealed">Advisory Vote 15: Repealed</option>
<option value="i;Times Blank Voted">Advisory Vote 15: Times Blank Voted</option>
<option value="j;Times Over Voted">Advisory Vote 15: Times Over Voted</option>
<option value="k;Bob Ferguson">Attorney General: Bob Ferguson</option>
<option value="l;Joshua B. Trumbull">Attorney General: Joshua B. Trumbull</option>
<option value="m;Times Blank Voted">Attorney General: Times Blank Voted</option>
<option value="n;Times Over Voted">Attorney General: Times Over Voted</option>
<option

{'Registered_Voters': 'a',
 'Total_Voters': 'b',
 'Advisory Vote 14: Maintained': 'c',
 'Advisory Vote 14: Repealed': 'd',
 'Advisory Vote 14: Times Blank Voted': 'e',
 'Advisory Vote 14: Times Over Voted': 'f',
 'Advisory Vote 15: Maintained': 'g',
 'Advisory Vote 15: Repealed': 'h',
 'Advisory Vote 15: Times Blank Voted': 'i',
 'Advisory Vote 15: Times Over Voted': 'j',
 'Attorney General: Bob Ferguson': 'k',
 'Attorney General: Joshua B. Trumbull': 'l',
 'Attorney General: Times Blank Voted': 'm',
 'Attorney General: Times Over Voted': 'n',
 'Attorney General: Write-In': 'o',
 'Commissioner of Public Lands: Hilary Franz': 'p',
 'Commissioner of Public Lands: Steve McLaughlin': 'q',
 'Commissioner of Public Lands: Times Blank Voted': 'r',
 'Commissioner of Public Lands: Times Over Voted': 's',
 'Commissioner of Public Lands: Write-In': 't',
 'Court of Appeals Judge Position 1: Michael J. Trickey': 'u',
 'Court of Appeals Judge Position 1: Times Blank Voted': 'v',
 'Court of Appeals J